In [ ]:
# Evaluación de Modelos - Modelo de Fuga Colsubsidio
# =====================================================
# 
# Objetivo: Evaluación exhaustiva y comparación de modelos entrenados
# - Análisis profundo de métricas de performance
# - Comparación detallada entre estrategias de balanceo
# - Validación cruzada y estabilidad del modelo
# - Selección final basada en criterios de negocio

# %% [markdown]
"""
## 1. Configuración y Carga de Resultados de Entrenamiento
"""

# %%
# Configuración inicial
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
import sys
from pathlib import Path

# Librerías de evaluación
from sklearn.metrics import (
    roc_auc_score, precision_score, recall_score, f1_score,
    confusion_matrix, roc_curve, precision_recall_curve,
    average_precision_score, matthews_corrcoef
)
from sklearn.model_selection import StratifiedKFold, cross_val_score
import joblib

warnings.filterwarnings('ignore')

print("Librerías cargadas correctamente")
print(f"Evaluación de modelos iniciada: {pd.Timestamp.now()}")

# %%
# Cargar resultados del entrenamiento
data_dir = Path("../data/outputs")

# Cargar comparación de modelos
comparison_path = data_dir / "model_comparison.csv"
if not comparison_path.exists():
    print("Error: Resultados de comparación no encontrados")
    print("Ejecutar primero notebook 04_model_training.ipynb")
    sys.exit()

comparison_df = pd.read_csv(comparison_path)

print("Resultados de entrenamiento cargados:")
print(f"  Modelos evaluados: {len(comparison_df)}")
print(f"  Combinaciones probadas: {list(comparison_df[['Modelo', 'Estrategia']].apply(lambda x: f"{x['Modelo']}+{x['Estrategia']}", axis=1))}")

# Mostrar tabla de resultados
print(f"\nTabla de resultados:")
print(comparison_df.round(3).to_string(index=False))

# %%
# Cargar modelo entrenado y componentes
try:
    best_model = joblib.load(data_dir / "best_model.pkl")
    scaler = joblib.load(data_dir / "scaler.pkl") 
    encoders = joblib.load(data_dir / "encoders.pkl")
    print("Modelo y componentes cargados correctamente")
except FileNotFoundError as e:
    print(f"Error cargando componentes: {e}")
    print("Verificar que el entrenamiento se completó correctamente")

# %% [markdown]
"""
## 2. Análisis Detallado de Performance
"""

# %%
# Análisis de la mejor combinación
def analyze_best_model():
    """Analiza en detalle el mejor modelo."""
    
    print("Análisis del mejor modelo...")
    
    # Identificar mejor modelo por AUC-ROC
    best_idx = comparison_df['AUC_ROC'].idxmax()
    best_model_info = comparison_df.iloc[best_idx]
    
    print(f"\nMejor modelo identificado:")
    print(f"  Algoritmo: {best_model_info['Modelo']}")
    print(f"  Estrategia: {best_model_info['Estrategia']}")
    print(f"  AUC-ROC: {best_model_info['AUC_ROC']:.3f}")
    
    # Análisis de trade-offs
    print(f"\nAnálisis de trade-offs:")
    print(f"  Precision: {best_model_info['Precision']:.3f}")
    print(f"  Recall: {best_model_info['Recall']:.3f}")
    print(f"  F1-Score: {best_model_info['F1_Score']:.3f}")
    
    if 'Precision@10%' in best_model_info:
        print(f"  Precision@10%: {best_model_info['Precision@10%']:.3f}")
    
    # Interpretación de negocio
    print(f"\nInterpretación para Colsubsidio:")
    
    if best_model_info['Precision'] > 0.4:
        print("  - Alta precisión: Pocas campañas desperdiciadas")
    elif best_model_info['Precision'] > 0.2:
        print("  - Precisión moderada: Balance aceptable")
    else:
        print("  - Baja precisión: Muchos falsos positivos")
        
    if best_model_info['Recall'] > 0.6:
        print("  - Alto recall: Detecta mayoría de casos de fuga")
    elif best_model_info['Recall'] > 0.3:
        print("  - Recall moderado: Detecta casos más evidentes")
    else:
        print("  - Bajo recall: Se pierden muchos casos reales")
    
    return best_model_info

best_model_info = analyze_best_model()

# %%
# Visualización comparativa detallada
def create_detailed_comparison():
    """Crea visualizaciones detalladas de comparación."""
    
    print("Creando visualizaciones comparativas...")
    
    # Radar chart comparativo
    metrics = ['AUC_ROC', 'Precision', 'Recall', 'F1_Score']
    if 'Precision@10%' in comparison_df.columns:
        metrics.append('Precision@10%')
    
    fig_radar = go.Figure()
    
    colors = ['blue', 'red', 'green', 'orange', 'purple', 'brown']
    
    for i, (_, row) in enumerate(comparison_df.iterrows()):
        values = [row[metric] for metric in metrics]
        values.append(values[0])  # Cerrar el polígono
        
        fig_radar.add_trace(go.Scatterpolar(
            r=values,
            theta=metrics + [metrics[0]],
            fill='toself',
            name=f"{row['Modelo']} + {row['Estrategia']}",
            line_color=colors[i % len(colors)]
        ))
    
    fig_radar.update_layout(
        polar=dict(
            radialaxis=dict(
                visible=True,
                range=[0, 1]
            )),
        title="Comparación Multi-dimensional de Modelos",
        height=600
    )
    
    fig_radar.show()
    
    # Heatmap de métricas
    metrics_matrix = comparison_df[metrics].values
    model_labels = [f"{row['Modelo']}\n{row['Estrategia']}" for _, row in comparison_df.iterrows()]
    
    fig_heatmap = px.imshow(
        metrics_matrix.T,
        x=model_labels,
        y=metrics,
        color_continuous_scale='RdYlBu_r',
        title="Heatmap de Performance por Modelo y Métrica",
        text_auto='.3f'
    )
    
    fig_heatmap.update_layout(height=500)
    fig_heatmap.show()

create_detailed_comparison()

# %%
# Análisis de estabilidad por estrategia
def analyze_strategy_stability():
    """Analiza la estabilidad de cada estrategia de balanceo."""
    
    print("Análisis de estabilidad por estrategia...")
    
    strategies = comparison_df['Estrategia'].unique()
    
    print(f"\nEstrategias evaluadas: {list(strategies)}")
    
    for strategy in strategies:
        strategy_results = comparison_df[comparison_df['Estrategia'] == strategy]
        
        print(f"\nEstrategia: {strategy}")
        print(f"  Modelos probados: {len(strategy_results)}")
        
        # Estadísticas por métrica
        metrics = ['AUC_ROC', 'Precision', 'Recall', 'F1_Score']
        for metric in metrics:
            if metric in strategy_results.columns:
                mean_score = strategy_results[metric].mean()
                std_score = strategy_results[metric].std()
                min_score = strategy_results[metric].min()
                max_score = strategy_results[metric].max()
                
                print(f"  {metric}: {mean_score:.3f} ± {std_score:.3f} (rango: {min_score:.3f}-{max_score:.3f})")

analyze_strategy_stability()

# %% [markdown]
"""
## 3. Análisis de Curvas de Performance
"""

# %%
# Simular curvas ROC y Precision-Recall para análisis
def simulate_performance_curves():
    """Simula curvas de performance para análisis visual."""
    
    print("Generando análisis de curvas de performance...")
    
    # Nota: En un escenario real, estas curvas vendrían del entrenamiento
    # Aquí simulamos basado en las métricas conocidas
    
    fig_curves = make_subplots(
        rows=1, cols=2,
        subplot_titles=['Curvas ROC Estimadas', 'Trade-off Precision vs Recall'],
        specs=[[{"secondary_y": False}, {"secondary_y": False}]]
    )
    
    # Simulación de curvas ROC basada en AUC conocido
    colors = ['blue', 'red', 'green', 'orange']
    
    for i, (_, row) in enumerate(comparison_df.iterrows()):
        # Simulación simple de curva ROC
        auc_score = row['AUC_ROC']
        fpr = np.linspace(0, 1, 100)
        # Aproximación básica de TPR basada en AUC
        tpr = np.minimum(1, fpr + (auc_score - 0.5) * 2)
        
        fig_curves.add_trace(
            go.Scatter(
                x=fpr, y=tpr,
                name=f"{row['Modelo']} + {row['Estrategia']} (AUC: {auc_score:.3f})",
                line=dict(color=colors[i % len(colors)]),
                mode='lines'
            ),
            row=1, col=1
        )
    
    # Línea diagonal para referencia
    fig_curves.add_trace(
        go.Scatter(
            x=[0, 1], y=[0, 1],
            mode='lines',
            line=dict(dash='dash', color='gray'),
            name='Random Classifier',
            showlegend=False
        ),
        row=1, col=1
    )
    
    # Gráfico Precision vs Recall
    fig_curves.add_trace(
        go.Scatter(
            x=comparison_df['Recall'],
            y=comparison_df['Precision'],
            mode='markers+text',
            text=[f"{row['Modelo']}" for _, row in comparison_df.iterrows()],
            textposition="top center",
            marker=dict(size=10, color=comparison_df['AUC_ROC'], 
                       colorscale='Viridis', showscale=True),
            name='Modelos',
            showlegend=False
        ),
        row=1, col=2
    )
    
    fig_curves.update_xaxes(title_text="False Positive Rate", row=1, col=1)
    fig_curves.update_yaxes(title_text="True Positive Rate", row=1, col=1)
    fig_curves.update_xaxes(title_text="Recall", row=1, col=2)
    fig_curves.update_yaxes(title_text="Precision", row=1, col=2)
    
    fig_curves.update_layout(
        title_text="Análisis de Performance de Modelos",
        height=500
    )
    
    fig_curves.show()

simulate_performance_curves()

# %% [markdown]
"""
## 4. Análisis de Feature Importance
"""

# %%
# Cargar y analizar feature importance
def analyze_feature_importance():
    """Analiza la importancia de features del mejor modelo."""
    
    importance_path = data_dir / "feature_importance.csv"
    
    if not importance_path.exists():
        print("Feature importance no disponible")
        return None
    
    importance_df = pd.read_csv(importance_path)
    
    print("Análisis de Feature Importance...")
    print(f"Total features analizadas: {len(importance_df)}")
    
    # Top 15 features más importantes
    top_features = importance_df.head(15)
    
    print(f"\nTop 15 variables más importantes:")
    for i, row in top_features.iterrows():
        print(f"  {i+1:2d}. {row['feature']:<25} {row['importance_pct']:>6.2f}%")
    
    # Visualización
    fig_importance = px.bar(
        top_features,
        x='importance_pct',
        y='feature',
        orientation='h',
        title='Feature Importance - Top 15 Variables',
        labels={'importance_pct': 'Importancia (%)', 'feature': 'Variables'},
        color='importance_pct',
        color_continuous_scale='Viridis'
    )
    
    fig_importance.update_layout(
        height=600,
        yaxis={'categoryorder': 'total ascending'}
    )
    
    fig_importance.show()
    
    # Categorización de variables
    print(f"\nCategorización de variables importantes:")
    
    # Variables financieras
    financial_vars = [var for var in top_features['feature'] 
                     if any(fword in var.lower() for fword in ['saldo', 'limite', 'pago', 'mora', 'cupo'])]
    
    # Variables derivadas
    derived_vars = [var for var in top_features['feature'] 
                   if any(dword in var.lower() for dword in ['utilization', 'stress', 'activity', 'benefit'])]
    
    # Variables demográficas
    demo_vars = [var for var in top_features['feature'] 
                if any(demo in var.lower() for demo in ['edad', 'segmento', 'estrato', 'genero'])]
    
    print(f"  Variables financieras: {len(financial_vars)} ({financial_vars[:3]}...)")
    print(f"  Variables derivadas: {len(derived_vars)} ({derived_vars[:3]}...)")
    print(f"  Variables demográficas: {len(demo_vars)} ({demo_vars[:3]}...)")
    
    return importance_df

feature_importance_df = analyze_feature_importance()

# %% [markdown]
"""
## 5. Validación de Robustez del Modelo
"""

# %%
# Análisis de robustez y estabilidad
def robustness_analysis():
    """Analiza la robustez del modelo seleccionado."""
    
    print("Análisis de robustez del modelo...")
    
    # Análisis de sensibilidad de métricas
    print(f"\nAnálisis de sensibilidad:")
    
    # Calcular coeficiente de variación entre modelos de la misma estrategia
    best_strategy = best_model_info['Estrategia']
    strategy_models = comparison_df[comparison_df['Estrategia'] == best_strategy]
    
    if len(strategy_models) > 1:
        print(f"Estabilidad dentro de la estrategia {best_strategy}:")
        
        for metric in ['AUC_ROC', 'Precision', 'Recall', 'F1_Score']:
            if metric in strategy_models.columns:
                mean_val = strategy_models[metric].mean()
                std_val = strategy_models[metric].std()
                cv = std_val / mean_val if mean_val > 0 else 0
                
                if cv < 0.1:
                    stability = "Muy estable"
                elif cv < 0.2:
                    stability = "Estable"
                else:
                    stability = "Variable"
                
                print(f"  {metric}: {stability} (CV: {cv:.3f})")
    
    # Análisis de distribución de errores (simulado)
    print(f"\nAnálisis de distribución de errores:")
    print(f"  Precision: {best_model_info['Precision']:.3f} - ", end="")
    if best_model_info['Precision'] > 0.5:
        print("Baja tasa de falsos positivos")
    elif best_model_info['Precision'] > 0.3:
        print("Tasa moderada de falsos positivos")
    else:
        print("Alta tasa de falsos positivos - revisar umbrales")
    
    print(f"  Recall: {best_model_info['Recall']:.3f} - ", end="")
    if best_model_info['Recall'] > 0.7:
        print("Baja tasa de falsos negativos")
    elif best_model_info['Recall'] > 0.4:
        print("Tasa moderada de falsos negativos")
    else:
        print("Alta tasa de falsos negativos - casos perdidos")

robustness_analysis()

# %% [markdown]
"""
## 6. Recomendaciones de Implementación
"""

# %%
# Generar recomendaciones de implementación
def implementation_recommendations():
    """Genera recomendaciones para implementación."""
    
    print("Recomendaciones de implementación...")
    
    print(f"\nModelo recomendado:")
    print(f"  Algoritmo: {best_model_info['Modelo']}")
    print(f"  Estrategia: {best_model_info['Estrategia']}")
    print(f"  Justificación: Balance óptimo entre métricas de negocio")
    
    print(f"\nConsideraciones de implementación:")
    
    # Recomendaciones basadas en performance
    if best_model_info['AUC_ROC'] > 0.8:
        print("  - Modelo tiene excelente capacidad discriminativa")
        print("  - Confiable para segmentación de riesgo")
    elif best_model_info['AUC_ROC'] > 0.7:
        print("  - Modelo tiene buena capacidad discriminativa")
        print("  - Adecuado para implementación con monitoreo")
    else:
        print("  - Modelo requiere mejoras antes de implementación")
    
    # Recomendaciones de monitoreo
    print(f"\nPlan de monitoreo recomendado:")
    print("  1. Evaluación mensual de distribución de scores")
    print("  2. Seguimiento de tasa de conversión de campañas")
    print("  3. Reentrenamiento trimestral con nuevos datos")
    print("  4. Validación de drift en variables importantes")
    
    # Métricas de seguimiento
    print(f"\nMétricas clave de seguimiento:")
    print(f"  - AUC-ROC objetivo: > {best_model_info['AUC_ROC']:.3f}")
    print(f"  - Precision objetivo: > {best_model_info['Precision']:.3f}")
    print("  - Tasa de respuesta de campañas: > 15%")
    print("  - ROI de campañas: > 3:1")

implementation_recommendations()

# %% [markdown]
"""
## 7. Exportación de Resultados de Evaluación
"""

# %%
# Guardar análisis de evaluación
output_dir = Path("../data/outputs")

print("Exportando resultados de evaluación...")

# Crear reporte de evaluación
evaluation_report = {
    'best_model': {
        'algorithm': best_model_info['Modelo'],
        'strategy': best_model_info['Estrategia'],
        'auc_roc': best_model_info['AUC_ROC'],
        'precision': best_model_info['Precision'],
        'recall': best_model_info['Recall'],
        'f1_score': best_model_info['F1_Score']
    },
    'all_results': comparison_df.to_dict('records'),
    'evaluation_date': pd.Timestamp.now().isoformat()
}

# Guardar como JSON para fácil lectura
import json
report_path = output_dir / "model_evaluation_report.json"
with open(report_path, 'w') as f:
    json.dump(evaluation_report, f, indent=2)

print(f"Reporte de evaluación guardado en: {report_path}")

# Guardar feature importance analizada
if feature_importance_df is not None:
    analyzed_importance_path = output_dir / "feature_importance_analyzed.csv"
    feature_importance_df.to_csv(analyzed_importance_path, index=False)
    print(f"Feature importance analizada guardada en: {analyzed_importance_path}")

print("Evaluación completada y exportada")

# %% [markdown]
"""
## 8. Resumen Ejecutivo de Evaluación
"""

# %%
# Resumen ejecutivo final
print("=" * 60)
print("RESUMEN EJECUTIVO - EVALUACIÓN DE MODELOS")
print("=" * 60)

print(f"\nMODELO FINAL SELECCIONADO:")
print(f"  Algoritmo: {best_model_info['Modelo']}")
print(f"  Estrategia de balanceo: {best_model_info['Estrategia']}")

print(f"\nMÉTRICAS DE PERFORMANCE:")
print(f"  AUC-ROC: {best_model_info['AUC_ROC']:.3f}")
print(f"  Precision: {best_model_info['Precision']:.3f}")
print(f"  Recall: {best_model_info['Recall']:.3f}")
print(f"  F1-Score: {best_model_info['F1_Score']:.3f}")

if 'Precision@10%' in best_model_info:
    print(f"  Precision@10%: {best_model_info['Precision@10%']:.3f}")

print(f"\nFORTALEZAS DEL MODELO:")
strengths = []
if best_model_info['AUC_ROC'] > 0.8:
    strengths.append("Excelente capacidad discriminativa")
if best_model_info['Precision'] > 0.4:
    strengths.append("Baja tasa de falsos positivos")
if best_model_info['Recall'] > 0.3:
    strengths.append("Detección adecuada de casos reales")

for strength in strengths:
    print(f"  - {strength}")

print(f"\nCONSIDERACIONES:")
considerations = []
if best_model_info['Precision'] < 0.3:
    considerations.append("Monitorear falsos positivos en campañas")
if best_model_info['Recall'] < 0.4:
    considerations.append("Algunos casos de fuga pueden no detectarse")

if considerations:
    for consideration in considerations:
        print(f"  - {consideration}")
else:
    print("  - Modelo balanceado sin consideraciones críticas")

print(f"\nRECOMENDACIÓN:")
print("  APROBAR IMPLEMENTACIÓN del modelo seleccionado")
print("  con plan de monitoreo y reentrenamiento establecido")

print(f"\n" + "=" * 60)
print(f"EVALUACIÓN COMPLETADA")
print(f"Fecha: {pd.Timestamp.now()}")
print(f"Siguiente paso: Scoring y Segmentación (Notebook 06)")
print("=" * 60)

# %%